In [1]:
import numpy as np
import pickle as pkl
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("Load_in_Great_Britain.csv")
df.columns = ['ds', 'y']
df.shape

(6395, 2)

In [3]:
quantiles = [0.015, 0.985]

params = {
    "n_lags": 24,
    "n_forecasts": 7,
    "n_changepoints": 20,
    "learning_rate": 0.01,
    "ar_layers": [32, 16, 16, 32],
    "epochs": 70,
    "batch_size": 64,
    "quantiles": quantiles,
}


m = NeuralProphet(**params)
m.set_plotting_backend('plotly-static')
set_log_level("ERROR")

In [4]:
df_train, df_test = m.split_df(df, valid_p=0.06, local_split=True)
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (6178, 2)
Test shape: (422, 2)


In [5]:
from tensorflow.keras.models import load_model
lstm_model = load_model("model_store/lstm_load.keras")
lstm_model.summary()

with open("model_store/best_order_load.pkl", "rb") as f:
    loaded_order = pkl.load(f)

with open("model_store/opt_no_states_load.pkl", "rb") as f:
    opt_states = pkl.load(f)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 40)                6720      
                                                                 
 dense_11 (Dense)            (None, 1)                 41        
                                                                 
Total params: 6761 (26.41 KB)
Trainable params: 6761 (26.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
from hmmlearn import hmm
from statsmodels.tsa.arima.model import ARIMA
from utils import softmax_weighting, get_mae_errors, get_mape_errors, get_rmse_errors
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(np.array(df['y']).reshape(-1,1))

def dynamic_ensemble_prediction(train, test):
    train_hmm = train.reshape(-1,1)
    train_hmm = scaler.transform(train_hmm)
    test_hmm = test.reshape(-1,1)
    test_hmm = scaler.transform(test_hmm)
    hmm_history = train_hmm
    history = np.array(train)

    predictions = []
    truth_values = []
    lstm_preds = []
    hmm_preds = []
    arima_preds = []

    for i in range(len(test)):
        print(f'{i+1}/{len(test)}')
        truth_values.append(test[i])
        # LSTM
        lstm_pred = lstm_model.predict(history[-24:].reshape(1,24))[0][0]
        lstm_preds.append(lstm_pred)
        # ARIMA
        arima_model = ARIMA(history, order=loaded_order)
        arima_fit = arima_model.fit()
        arima_pred = arima_fit.forecast(steps=1)[0]
        arima_preds.append(arima_pred)
        # HMM
        hmm_model = hmm.GaussianHMM(n_components=opt_states, covariance_type='diag', tol=0.0001, n_iter=100)
        hmm_model.fit(hmm_history)
        hidden_states = hmm_model.predict(hmm_history)
        last_hidden_state = hidden_states[-1]
        next_state_probs = hmm_model.transmat_[last_hidden_state]
        predicted_state = np.argmax(next_state_probs)
        predicted_value = hmm_model.means_[predicted_state][0]
        hmm_pred = scaler.inverse_transform(np.array(predicted_value).reshape(-1,1))[0][0]
        hmm_preds.append(hmm_pred)

        #Error Measurement
        arima_error = get_rmse_errors(arima_preds, truth_values)
        hmm_error = get_rmse_errors(hmm_preds, truth_values)
        lstm_error = get_rmse_errors(lstm_preds, truth_values) 
        weights = softmax_weighting(arima_error, lstm_error, hmm_error,1) # Weighting algorithm
        predictions.append(weights[0]*arima_pred + weights[1]*lstm_pred + weights[2]*hmm_pred)
        history = np.append(history,test[i])
        
        if i != len(test)-1:
            hmm_history = np.append(hmm_history,test_hmm[i]).reshape(-1,1)
    
    return predictions, arima_preds, hmm_preds, lstm_preds
        

In [ ]:
de_preds, arima_preds, hmm_preds, lstm_preds = dynamic_ensemble_prediction(np.array(df_train.y), np.array(df_test.y))

1/422
1/1 [==============================] - 0s 24ms/step
2/422
1/1 [==============================] - 0s 25ms/step
3/422
1/1 [==============================] - 0s 22ms/step
4/422
1/1 [==============================] - 0s 28ms/step
5/422
1/1 [==============================] - 0s 23ms/step
6/422
1/1 [==============================] - 0s 24ms/step
7/422
1/1 [==============================] - 0s 22ms/step
8/422
1/1 [==============================] - 0s 23ms/step
9/422
1/1 [==============================] - 0s 16ms/step
10/422
1/1 [==============================] - 0s 24ms/step
11/422
1/1 [==============================] - 0s 31ms/step
12/422
1/1 [==============================] - 0s 20ms/step
13/422
1/1 [==============================] - 0s 27ms/step
14/422
1/1 [==============================] - 0s 14ms/step
15/422
1/1 [==============================] - 0s 15ms/step
16/422
1/1 [==============================] - 0s 28ms/step
17/422
1/1 [==============================] - 0s 24ms/step
18/422

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math
print('Dynamic Ensemble')
print(f'R2 Score : {r2_score(df_test.y,de_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,de_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,de_preds)}')
print('ARIMA')
print(f'R2 Score : {r2_score(df_test.y,arima_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,arima_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,arima_preds)}')
print('HMM')
print(f'R2 Score : {r2_score(df_test.y,hmm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,hmm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,hmm_preds)}')
print('LSTM')
print(f'R2 Score : {r2_score(df_test.y,lstm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y, lstm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,lstm_preds)}')

Dynamic Ensemble


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], mode='lines', name='Real Data'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_preds, mode='lines', name='Proposed Method'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_preds, mode='lines', name='ARIMA'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_preds, mode='lines', name='HMM'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_preds, mode='lines', name='LSTM'))
fig.add_trace(go.Scatter(x=df_train['ds'], y=df_train['y'], mode='lines', name='Training'))
fig.show()